sudo scp access.log.1 litinsky@192.168.10.221:/home/litinsky/dev/log_analysis/access.log.1

pd.set_option('display.max_colwidth', 200) default: 50

In [9]:
import re
import gzip
import hashlib
from pathlib import Path
from datetime import datetime
from typing import Optional

import pandas as pd

from log_info import update_log_df, DEFAULT_CONFIG


pd.set_option('display.max_colwidth', 200)

In [10]:
for key, value in DEFAULT_CONFIG.items():
    print(key, ':', value)

log_dir : ../nginx_t
pickle_file : logs_df.pkl
hashes_file : hashes.txt
reports_dir : reports
log_file_name_glob_pattern : access.log*
log_arc_file_name_re_pattern : .+\.gz$
log_entry_pattern : ^(\S+?)\s(\S+?)\s(\S+?)\s(\[.+?\])\s(".+?")\s(.+?)\s(.+?)\s(".+?")\s(".+?")\s(".+?")$
log_entry_fields : ['ip_from', 'domain', '_1', 'timestamp', 'request', 'response_code', 'time', 'ref', 'app', '_2']
log_dataframe_fields : ['ip_from', 'domain', '_1', 'timestamp', 'request', 'response_code', 'time', 'ref', 'app', '_2', 'internal_request', 'resource']
filter_match : []
filter_not_match : []
filter_in : []
filter_not_in : []


In [11]:
config = DEFAULT_CONFIG

In [13]:
log_df: pd.DataFrame = update_log_df(config=config)
log_df.describe()

,ip_from,domain,_1,timestamp,request,response_code,time,ref,app,_2
count,1068431,1068431,1068431,1068431,1068431,1068431,1068431,1068431,1068431,1068431
unique,502,222,2,86303,597,9,574,14,90,5
top,10.11.1.9,2242.lnsigo.mipt.ru,-,[10/Dec/2018:17:01:27 +0300],"""GET /botad558a10-8a7e-48ac-95a2-3cc9aa318dbd/getUpdates HTTP/1.1""",200,26,"""-""","""python-requests/2.16.5""","""-"""
freq,783721,1034387,1068430,41,700006,1064231,1061438,1067744,700006,1068424


In [19]:
log_filteres = log_df[log_df['ip_from'].isin(['files.deeppavlov.ai'])]
log_filteres.describe()

,ip_from,domain,_1,timestamp,request,response_code,time,ref,app,_2
count,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0


In [22]:
log_filtered = log_df[(log_df['domain'].str.match(r'files.deeppavlov.ai')==True) & 
                      (log_df.request.str.match(r'^"GET /.+md5 HTTP/1\.1"$')==False) & 
                      (log_df.request.str.match(r'^"GET /\s+HTTP/1\.1"$')==False) & 
                      (log_df.request.str.match(r'^"GET /robots.txt HTTP/1\.1"$')==False)]

log_filtered.describe()

,ip_from,domain,_1,timestamp,request,response_code,time,ref,app,_2
count,2131,2131,2131,2131,2131,2131,2131,2131,2131,2131
unique,80,1,1,1869,149,3,206,6,28,1
top,10.11.1.101,files.deeppavlov.ai,-,[10/Dec/2018:17:20:27 +0300],"""GET /faq/school/faq_school.csv HTTP/1.1""",404,29257,"""-""","""python-requests/2.19.1""","""-"""
freq,533,2131,2131,5,121,2127,121,2124,1856,2131


In [19]:
log_filtered.head(3)

,ip_from,domain,_1,timestamp,request,response_code,time,ref,app,_2
2448,10.11.1.56,files.deeppavlov.ai,-,[19/Dec/2018:06:29:48 +0300],"""GET /deeppavlov_data/paraphrase_ident_paraphraser_tuned.tar.gz HTTP/1.1""",404,5388678,"""-""","""python-requests/2.19.1""","""-"""
4412,10.11.1.56,files.deeppavlov.ai,-,[19/Dec/2018:06:33:27 +0300],"""GET /deeppavlov_data/elmo_news_wmt11-16-simple_reduce_para_pretrain_fine_tuned_ep1.tar.gz HTTP/1.1""",404,694715762,"""-""","""python-requests/2.19.1""","""-"""
30296,10.11.1.56,files.deeppavlov.ai,-,[19/Dec/2018:07:21:33 +0300],"""GET /deeppavlov_data/paraphraser_elmo_ft_pre_1_model.tar.gz HTTP/1.1""",404,9913204,"""-""","""python-requests/2.19.1""","""-"""


In [23]:
log_filtered.loc[:, 'ttimestamp'] = log_filtered.timestamp.transform(lambda x: datetime.strptime(x, '[%d/%b/%Y:%H:%M:%S %z]'))
log_filtered.loc[:, 'timestamp'] = log_filtered.timestamp.transform(lambda x: datetime.strptime(x, '[%d/%b/%Y:%H:%M:%S %z]'))
log_filtered.describe()

,ip_from,domain,_1,timestamp,request,response_code,time,ref,app,_2,ttimestamp
count,2131,2131,2131,2131,2131,2131,2131,2131,2131,2131,2131
unique,80,1,1,1869,149,3,206,6,28,1,1869
top,10.11.1.101,files.deeppavlov.ai,-,2018-12-10 17:20:27+03:00,"""GET /faq/school/faq_school.csv HTTP/1.1""",404,29257,"""-""","""python-requests/2.19.1""","""-""",2018-12-10 17:20:27+03:00
freq,533,2131,2131,5,121,2127,121,2124,1856,2131,5


In [65]:
log_filtered.head(3)

,ip_from,domain,_1,timestamp,request,response_code,time,ref,app,_2
2448,10.11.1.56,files.deeppavlov.ai,-,2018-12-19 06:29:48+03:00,"""GET /deeppavlov_data/paraphrase_ident_paraphraser_tuned.tar.gz HTTP/1.1""",404,5388678,"""-""","""python-requests/2.19.1""","""-"""
4412,10.11.1.56,files.deeppavlov.ai,-,2018-12-19 06:33:27+03:00,"""GET /deeppavlov_data/elmo_news_wmt11-16-simple_reduce_para_pretrain_fine_tuned_ep1.tar.gz HTTP/1.1""",404,694715762,"""-""","""python-requests/2.19.1""","""-"""
30296,10.11.1.56,files.deeppavlov.ai,-,2018-12-19 07:21:33+03:00,"""GET /deeppavlov_data/paraphraser_elmo_ft_pre_1_model.tar.gz HTTP/1.1""",404,9913204,"""-""","""python-requests/2.19.1""","""-"""


In [72]:
# log_group = log_filtered.groupby(['request', 'ip_from'])
log_group = log_filtered.groupby(['request'])

In [82]:
reqs = log_filtered.request.drop_duplicates()
reqs

2448                                          "GET /deeppavlov_data/paraphrase_ident_paraphraser_tuned.tar.gz HTTP/1.1"
4412               "GET /deeppavlov_data/elmo_news_wmt11-16-simple_reduce_para_pretrain_fine_tuned_ep1.tar.gz HTTP/1.1"
30296                                            "GET /deeppavlov_data/paraphraser_elmo_ft_pre_1_model.tar.gz HTTP/1.1"
30761                                          "GET /deeppavlov_data/classifiers/yahoo_convers_vs_info.tar.gz HTTP/1.1"
30769                                              "GET /embeddings/yahooo-sber-questions_epoches_n_15.tar.gz HTTP/1.1"
31025                                                                  "GET /datasets/snips_intents/train.csv HTTP/1.1"
31036                                               "GET /deeppavlov_data/classifiers/intents_snips_v7.tar.gz HTTP/1.1"
31303                                               "GET /deeppavlov_data/classifiers/intents_snips_v8.tar.gz HTTP/1.1"
33970                                   

In [73]:
log_group.count()

,ip_from,domain,_1,timestamp,response_code,time,ref,app,_2
request,,,,,,,,,
"""GET /datasets HTTP/1.1""",1,1,1,1,1,1,1,1,1
"""GET /datasets/ HTTP/1.1""",2,2,2,2,2,2,2,2,2
"""GET /datasets/UD2.0_source/ HTTP/1.1""",1,1,1,1,1,1,1,1,1
"""GET /datasets/UD2.0_source/en.tar.gz HTTP/1.1""",14,14,14,14,14,14,14,14,14
"""GET /datasets/UD2.0_source/ru_syntagrus.tar.gz HTTP/1.1""",13,13,13,13,13,13,13,13,13
"""GET /datasets/ag_news_data.tar.gz HTTP/1.1""",18,18,18,18,18,18,18,18,18
"""GET /datasets/convers_vs_info_data.tar.gz HTTP/1.1""",1,1,1,1,1,1,1,1,1
"""GET /datasets/coreference_resolution/char_vocab.russian.txt HTTP/1.1""",3,3,3,3,3,3,3,3,3
"""GET /datasets/coreference_resolution/pretrained_model.zip HTTP/1.1""",3,3,3,3,3,3,3,3,3
